In [5]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [6]:
! pip install ffmpeg sk-video

In [7]:
import os
!mkdir s1
!mkdir s1_align
os.listdir()

#################### DOWNLOAD AND UNZIP FILE SAVED IN DRIVE ####################

!pip install -U -q PyDrive

s1_folder_id = '1B8cIYz6ljEbjYapG6EW1NrKHgcpZ2HzD'
s1_align_folder_id = '1swcuyFY-ZdNgFBou5GiigfEXsURcPO63'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

mkdir: cannot create directory ‘s1’: File exists
mkdir: cannot create directory ‘s1_align’: File exists


In [0]:
# %cd training
%cd '/content/s1'
!pwd

file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(s1_folder_id)}).GetList()

for file1 in sorted(file_list, key = lambda x: x['title']):
    print('Downloading {} from GDrive'.format(file1['title'])) # i+1, len(file_list)))
    file1.GetContentFile(file1['title'])

/content/s1
/content/s1


In [0]:
%cd '/content/s1_align'
!pwd

file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(s1_align_folder_id)}).GetList()

for file1 in sorted(file_list, key = lambda x: x['title']):
    print('Downloading {} from GDrive'.format(file1['title'])) # i+1, len(file_list)))
    file1.GetContentFile(file1['title'])

/content/s1_align
/content/s1_align


In [0]:
# check directories
%cd '/content/'
# os.listdir('/content'), os.listdir('/content/training'), os.listdir('/content/labels')

/content


## Model + IO


In [5]:
! rm -r MathResearchQHSS
! git clone -b lipreading https://github.com/jchen42703/MathResearchQHSS.git
%cd MathResearchQHSS

Cloning into 'MathResearchQHSS'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 298 (delta 89), reused 0 (delta 0), pack-reused 151
Receiving objects: 100% (298/298), 20.78 MiB | 9.47 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/MathResearchQHSS


In [6]:
!pip install -r requirements.txt
%cd '/content/'

/content


In [3]:
from MathResearchQHSS.lipreading.io.generator import FrameGenerator
from MathResearchQHSS.lipreading.models import LipNet
from MathResearchQHSS.lipreading.io.io_utils import get_list_IDs
import os
from glob import glob
import skvideo

n_epochs = 200

s1_path = '/content/s1/'
s1_align_path = '/content/s1_align/'

list_IDs = get_list_IDs(s1_path, val_split = 0.8)
data_dirs = [s1_path, s1_align_path]
train_gen = FrameGenerator(list_IDs['train'], data_dirs, batch_size = 1)
val_gen = FrameGenerator(list_IDs['val'], data_dirs, batch_size = 1)
# gen.__getitem__(1)

Using TensorFlow backend.


In [4]:
inp, out = train_gen.__getitem__(1)
inp['the_input'].shape

(1, 75, 576, 720, 3)

In [0]:
lipnet = LipNet()

In [0]:
from keras.optimizers import Adam
adam = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [10]:
lipnet.model.fit_generator(generator = train_gen, epochs = n_epochs, max_queue_size = 1, workers = 2, use_multiprocessing = True)

Epoch 1/200


ResourceExhaustedError: ignored

In [0]:
model.save_weights('train_weights_'+str(n_epochs)+'.h5')